In [1]:
from qiskit import QuantumCircuit
from qiskit_aer.primitives import Estimator
import matplotlib.pyplot as plt
import numpy as np
from qiskit.quantum_info import Statevector, random_clifford

from tomography import Random_Measurements
from tomography import NearSparseTomography 

from time import time 

In [2]:
NQs = [ 5 ]
shots = 1000
simulator_ideal=Estimator(backend_options={'shots':shots,
                                            'method':"stabilizer",},
                            transpile_options={'optimization_level':0},
                            abelian_grouping=True, ) 

In [3]:
MC = 10
Fids = np.zeros( [len(NQs),MC] )
num_meas = [15, 40, 100, 200, 350]

t1 = time() 
for i, NQ in enumerate(NQs):
    
    d = 2**NQ

    for j in range(MC): 
        RM = Random_Measurements(NQ)

        psi_circ = random_clifford( NQ ).to_circuit()
        psi_th = np.array( Statevector(psi_circ) )

        RM.RandomMeasurements( num_meas[i], psi_circ, simulator_ideal )

        phi_in = np.random.rand(d) + 1j * np.random.rand(d)
        phi_in = phi_in / np.linalg.norm(phi_in)

        psi_out, results = NearSparseTomography( phi_in, RM )

        fid = np.abs( np.vdot( psi_th, psi_out ) )**2 
        
        Fids[i,j] = fid
t2 = time()
t2-t1 

21.050097227096558

In [4]:
Fids.round(2)

array([[0.11, 0.08, 0.21, 0.12, 0.07, 0.01, 0.01, 0.04, 0.03, 0.03]])

In [5]:
np.mean( Fids, axis=1 )

array([0.07179777])

In [6]:
# 2.6348488330841064
# 3.2828805446624756
# 5.260049104690552
# 21.050097227096558